In my (limited) experience in the financial ML, it is better to start simple. 

As this is my very first notebook in this competition, I decided to use a linear model as my baseline. By using a linear model, we can effectively see how each anonymized feature is associated with the target.

I use the following parquet file to fit everything within this notebook.

[⏫ Fast Data Loading and Low Mem with Parquet Files](https://www.kaggle.com/robikscube/fast-data-loading-and-low-mem-with-parquet-files)

So let's get it started!

# Libraries

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import gc
import pathlib
from tqdm.auto import tqdm
import joblib
import pathlib
import json
import glob
import time
import datetime
from scipy import stats
from multiprocessing import Pool, cpu_count

# models
from sklearn import linear_model
from xgboost import XGBRegressor
import lightgbm as lgb

# visualize
import matplotlib.pyplot as plt
import matplotlib.style as style
from matplotlib_venn import venn2, venn3
import seaborn as sns
from matplotlib import pyplot
from matplotlib.ticker import ScalarFormatter
sns.set_context("talk")
style.use('seaborn-colorblind')

import warnings
warnings.simplefilter('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Load data

In [ ]:
# train = pd.read_parquet('../input/ubiquant-parquet/train.parquet')
train = pd.read_parquet('../input/ubiquant-parquet/train_low_mem.parquet')

print(train.shape)
train.head()

In [ ]:
ss = pd.read_csv('/kaggle/input/ubiquant-market-prediction/example_sample_submission.csv')
print(ss.shape)
ss.head()

# Modeling

In [ ]:
features = train.columns[train.columns.str.startswith('f_')].values.tolist()
print(len(features), features)

In [ ]:
# fit a linear model
model = linear_model.Ridge()
model.fit(train[features], train['target'])

In [ ]:
# coef
feature_importance = pd.DataFrame()
feature_importance['features'] = features
feature_importance['weight'] = model.coef_.ravel()

feature_importance.sort_values(by=['weight'], ascending=False).style.background_gradient(cmap='viridis')

# Submission

In [ ]:
%%time
import ubiquant

env = ubiquant.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission
for (test_df, sample_prediction_df) in iter_test:    
    sample_prediction_df['target'] = model.predict(test_df[features])  # make your predictions here
    env.predict(sample_prediction_df)   # register your predictions